In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import preprocessing from sklearn
from sklearn import preprocessing
# load dataset
pathDataset = "../Dataset/results.xlsx"
original_dataset  = pd.read_excel(pathDataset)

#Format date column to date time
#dataset['date'] = pd.to_datetime(X.date)
original_dataset['date'] =  pd.to_datetime(original_dataset['date'], format='%Y/%m/%d')
original_dataset['date'] =  pd.to_datetime(original_dataset['date'], format='%Y%-m-%d')
#original_dataset.dtypes
#original_dataset.describe()

#Add target column to our dataset 
#Create a new function:
def add_target(home, away):
    data = "equal"
    if home > away : 
        data = "Home winner"
    elif away > home : 
        data = "away winner"
    return data
original_dataset['target'] = original_dataset.apply(lambda x: add_target(x['home_score'], x['away_score']), axis=1)


# Split new dataset in Train and Test sub-sets
tarining_dataset = original_dataset[0:20948]
test_dataset = original_dataset[20949:38685]


# Training sub-dataset
X_train = tarining_dataset.iloc[:, 0:9]
Y_train = tarining_dataset.iloc[:, 8]

# Test sub-dataset
X_test = test_dataset.iloc[:, 0:9]
Y_test = test_dataset.iloc[:, 8]

#Display 3 first rows
original_dataset.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,target
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,equal
1,1872-11-30,England,Scotland,4,2,Friendly,London,England,Home winner
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,Home winner
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,equal
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,Home winner


In [2]:
# limit to categorical data using df.select_dtypes()
columns_categorical_subset = original_dataset.select_dtypes(include=[object])
columns_categorical_subset.head(3)

,home_team,away_team,tournament,city,country,target
0,Scotland,England,Friendly,Glasgow,Scotland,equal
1,England,Scotland,Friendly,London,England,Home winner
2,Scotland,England,Friendly,Glasgow,Scotland,Home winner


In [3]:
# check original shape
columns_categorical_subset.shape
# Y_test3 = to_categorical(y_test3, num_classes)

(38685, 6)

In [4]:
# view columns using df.columns
columns_categorical_subset.columns

Index(['home_team', 'away_team', 'tournament', 'city', 'country', 'target'], dtype='object')

In [15]:
# TODO: create a LabelEncoder object and fit it to each feature in X
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
le

# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
final_dataset = columns_categorical_subset.apply(le.fit_transform)
#final_dataset.head()

In [16]:
# TODO: create a OneHotEncoder object, and fit it to all of X

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(final_dataset)

# 3. Transform
onehotlabels = enc.transform(final_dataset).toarray()
onehotlabels
# Explication: 
# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [7]:
type(onehotlabels)

numpy.ndarray

In [17]:
final_dataset_values = final_dataset
#X = final_dataset_values[:,0:7]
#Y = final_dataset_values[:,8]

In [20]:
# Split new dataset in Train and Test sub-sets
tarining_dataset = onehotlabels[0:20948]
test_dataset = onehotlabels[20949:38685]

# Training sub-dataset
X_train = tarining_dataset[:, 0:9]
Y_train = tarining_dataset[:, 8]
#print(X_train.head(3))
print("###################################")

# Test sub-dataset
X_test = test_dataset[:, 0:8]
Y_test = test_dataset[:, 8]
#print(X_test.head(3))
onehotlabels.shape

###################################


(38685, 2647)

#CNN

In [ ]:
#from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten
#print(X_train.ndim)
#model2 = Sequential()
#model2.add(Conv2D(2, (3,3), padding='same', input_shape=X_train.shape[:1]))
#model2.add(Activation('relu'))
##model2.add(Conv2D(32, (3,3)))
#model2.add(Activation('relu'))
#model2.add(MaxPooling2D(pool_size=(2,2)))
#model2.add(Dropout(0.25))
#model2.add(Conv2D(64, (3,3), padding='same'))
#model2.add(Activation('relu'))
#model2.add(Conv2D(64, (3, 3)))
#model2.add(Activation('relu'))
#model2.add(MaxPooling2D(pool_size=(2,2)))
#model2.add(Dropout(0.25))
#model2.add(Flatten())
#model2.add(Dense(512))
#model2.add(Activation('relu'))
#model2.add(Dropout(0.5))
#model2.add(Dense(num_classes))
#model2.add(Activation('softmax'))

#RNN

In [ ]:
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Embedding(20000,128))
model.add(Dense(2647,activation='relu'))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Train .....")
model.fit(X_train, Y_train, batch_size=35, epochs=2, verbose=1, validation_data=(X_test, Y_test))

print("Evaluate .....")
score = model.evaluate(X_test, Y_test, batch_size=35)

print('Test score:', score)
print("Accuracy: %.2f%%" % (score[1]*100))

Train .....
Train on 20948 samples, validate on 17736 samples
Epoch 1/2
 1925/20948 [=>............................] - ETA: 3:26 - loss: 0.0865 - acc: 0.9870

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

#Predict class
predict = model.predict(X_test, batch_size=35)
classes = model.predict_classes(Y_test,batch_size=35)

#print(predict[3,])
print("########################")
print(classes)
print("########################")
#print(Y_test)
confusion_matrix(Y_test, predict)

#print(X_test[3,])
#print(final_dataset_values[3,])
#print(original_dataset[3,])
#print("########################")


########################
[[0]
 [0]
 [0]
 ..., 
 [0]
 [0]
 [0]]
########################


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [12]:
from keras.models import load_model
model.save('rnn_v1.h5')
rnn_v1 = load_model('rnn_v1.h5')